In [ ]:

def inverse_warp_view_2_to_1(heatmaps2, depths2, depths1, c2Tc1s, K1, K2, inv_thetas1, thetas2, depth_thresh=0.5, get_warped_depth=False):
    # compute warping xy coordinate from view1 to view2
    # Args
    #   depths1: [B,H,W,1] tf.float32
    #   c2Tc1s: [B,4,4] tf.float32
    #   K1,K2: [B,3,3] tf.float32
    #   inv_thetas1: [B,3,3] tf.float32
    #   thetas2: [B,3,3] tf.float32
    # Return
    #   heatmaps1w : [B,H,W,1] tf.float32 warped heatmaps from camera2 to camera1
    #   visible_masks1 : [B,H,W,1] tf.float32 visible masks on camera1
    #   xy_u2 : [B,H,W,2] tf.float32 xy-coords-maps from camera1 to camera2

    def norm_meshgrid(batch, height, width, is_homogeneous=True):
        """Construct a 2D meshgrid.

        Args:
            batch: batch size
            height: height of the grid
            width: width of the grid
            is_homogeneous: whether to return in homogeneous coordinates
        Returns:
            x,y grid coordinates [batch, 2 (3 if homogeneous), height, width]
        """
        x_t = tf.matmul(tf.ones(shape=tf.stack([height, 1])),
                      tf.transpose(tf.expand_dims(
                          tf.linspace(-1.0, 1.0, width), 1), [1, 0]))
        y_t = tf.matmul(tf.expand_dims(tf.linspace(-1.0, 1.0, height), 1),
                      tf.ones(shape=tf.stack([1, width])))
        if is_homogeneous:
            ones = tf.ones_like(x_t)
            coords = tf.stack([x_t, y_t, ones], axis=0)
        else:
            coords = tf.stack([x_t, y_t], axis=0)
        coords = tf.tile(tf.expand_dims(coords, 0), [batch, 1, 1, 1])
        return coords

    def norm_xy_coords(xyz, height, width):
        # xyz: [B,>=3,N], tf.float32 xyz[:,0] = x, xyz[:,1]=y
        # suppose 0<=x<width, 0<=y<height
        # outputs range will be [-1,1]
        x_t = tf.slice(xyz, [0,0,0], [-1,1,-1])
        y_t = tf.slice(xyz, [0,1,0], [-1,1,-1])
        z_t = tf.slice(xyz, [0,2,0], [-1,-1,-1])    
        x_t = 2 * (x_t / tf.cast(width-1,tf.float32)) - 1.0
        y_t = 2 * (y_t / tf.cast(height-1, tf.float32)) - 1.0
        n_xyz = tf.concat([x_t, y_t, z_t], axis=1)
        return n_xyz

    def unnorm_xy_coords(xyz, height, width):
        # xyz: [B,>=3,N], tf.float32 xyz[:,0] = x, xyz[:,1]=y
        # suppose -1<=x<=1, -1<=y<=1
        # outputs range will be [0,width) or [0,height)
        
        x_t = tf.slice(xyz, [0,0,0], [-1,1,-1])
        y_t = tf.slice(xyz, [0,1,0], [-1,1,-1])
        z_t = tf.slice(xyz, [0,2,0], [-1,-1,-1])    

        x_t = (x_t+1.0) * 0.5 * tf.cast(width-1, tf.float32)
        y_t = (y_t+1.0) * 0.5 * tf.cast(height-1, tf.float32)

        u_xyz = tf.concat([x_t, y_t, z_t], axis=1)
        return u_xyz

    with tf.name_scope('WarpCoordinates'):
        if K2 is None:
            K2 = K1 # use same intrinsic matrix

        eps = 1e-6
        batch_size = tf.shape(depths1)[0]
        height1 = tf.shape(depths1)[1]
        width1 = tf.shape(depths1)[2]

        inv_K1 = tf.matrix_inverse(K1)
        right_col = tf.zeros([batch_size,3,1], dtype=tf.float32)
        bottom_row = tf.tile(tf.constant([0,0,0,1], dtype=tf.float32, shape=[1,1,4]), [batch_size,1,1])
        K2_4x4 = tf.concat([K2, right_col], axis=2)
        K2_4x4 = tf.concat([K2_4x4, bottom_row], axis=1)

        xy_n1 = norm_meshgrid(batch_size, height1, width1) # [B,3,H,W]
        xy_n1 = tf.reshape(xy_n1, [batch_size, 3, -1]) # [B,3,N], N=H*W

        # Inverse inplane transformation on camera1
        if inv_thetas1 is not None:
            xy_n1 = tf.matmul(inv_thetas1, xy_n1)
            z_n1 = tf.slice(xy_n1, [0,2,0],[-1,1,-1])
            xy_n1 = xy_n1 / (z_n1+eps)

        # SE(3) transformation : pixel1 to camera1
        Z1 = tf.reshape(depths1, [batch_size, 1, -1])
        xy_u1 = unnorm_xy_coords(xy_n1, height=height1, width=width1)
        XYZ1 = tf.matmul(inv_K1, xy_u1) * Z1
        ones = tf.ones([batch_size, 1, height1*width1])
        XYZ1 = tf.concat([XYZ1, ones], axis=1)

        # SE(3) transformation : camera1 to camera2 to pixel2
        proj_T = tf.matmul(K2_4x4, c2Tc1s)
        xyz2 = tf.matmul(proj_T, XYZ1)
        z2 = tf.slice(xyz2, [0,2,0], [-1,1,-1])
        reproj_depths = tf.reshape(z2, [batch_size, 1, height1, width1])
        reproj_depths = tf.transpose(reproj_depths, perm=[0,2,3,1]) # [B,H,W,1]

        xy_u2 = tf.slice(xyz2, [0,0,0],[-1,3,-1]) / (z2+eps)

        # Inplane transformation on camera2
        if thetas2 is not None:
            xy_n2 = norm_xy_coords(xy_u2, height1, width1)
            xy_n2 = tf.matmul(thetas2, xy_n2)
            z_n2 = tf.slice(xy_n2,[0,2,0], [-1,1,-1])
            xy_n2 = xy_n2 / (z_n2+eps)
            xy_u2 = unnorm_xy_coords(xy_n2, height1, width1)
        xy_u2 = tf.slice(xy_u2, [0,0,0],[-1,2,-1]) # discard third dim
        xy_u2 = tf.reshape(xy_u2, [batch_size, 2, height1, width1])
        xy_u2 = tf.transpose(xy_u2, perm=[0, 2, 3, 1]) # [B,H,W,2]

        heatmaps1w, depths1w = bilinear_sampling(heatmaps2, xy_u2, depths2) # it is not correct way to check depth consistency but it works well practically 
        visible_masks = get_visibility_mask(xy_u2)
        camfront_masks = tf.cast(tf.greater(reproj_depths, tf.zeros((), reproj_depths.dtype)), tf.float32)
        nonocc_masks = tf.cast(tf.less(tf.squared_difference(depths1w, depths1), depth_thresh**2), tf.float32)
        visible_masks = visible_masks * camfront_masks * nonocc_masks # take logical_and
        heatmaps1w = heatmaps1w * visible_masks

        if get_warped_depth:
            return heatmaps1w, visible_masks, xy_u2, depths1w
        else:
            return heatmaps1w, visible_masks, xy_u2
